In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

import pydicom

from pymedphys.gamma import gamma_shell, GammaInternalFixedOptions, calculate_min_dose_difference, convert_to_percent_pass

In [ ]:
DATA_DIRECTORY = "../../../tests/data/gamma/agnew_mcgarry_images"
glob(DATA_DIRECTORY)

In [ ]:
REF_VMAT_1mm = os.path.abspath(os.path.join(
    DATA_DIRECTORY, 'H&N_VMAT_Reference_1mmPx.dcm'))
EVAL_VMAT_1mm = os.path.abspath(os.path.join(
    DATA_DIRECTORY, 'H&N_VMAT_Evaluated_1mmPx.dcm'))

REF_VMAT_0_25mm = os.path.abspath(os.path.join(
    DATA_DIRECTORY, 'H&N_VMAT_Reference_0_25mmPx.dcm'))
EVAL_VMAT_0_25mm = os.path.abspath(os.path.join(
    DATA_DIRECTORY, 'H&N_VMAT_Evaluated_0_25mmPx.dcm'))

In [ ]:
def load_dose_from_dicom(dcm):
    pixels = dcm.pixel_array
    dose = pixels * dcm.DoseGridScaling

    return dose


def load_yx_from_dicom(dcm):
    resolution = np.array(
        dcm.PixelSpacing).astype(float)
    dx = resolution[0]
    x = (
        dcm.ImagePositionPatient[0] +
        np.arange(0, dcm.Columns * dx, dx))

    dy = resolution[1]
    y = (
        dcm.ImagePositionPatient[1] +
        np.arange(0, dcm.Rows * dy, dy))

    return y, x

In [ ]:
dcm_ref = pydicom.read_file(REF_VMAT_0_25mm)
dcm_eval = pydicom.read_file(EVAL_VMAT_0_25mm)

coords_reference = load_yx_from_dicom(dcm_ref)
dose_reference = load_dose_from_dicom(dcm_ref)

coords_evaluation = load_yx_from_dicom(dcm_eval)
dose_evaluation = load_dose_from_dicom(dcm_eval)

In [ ]:
%%timeit

dose_thresholds = 0.5
distance_thresholds = 0.5

gamma = gamma_shell(
    coords_reference, dose_reference,
    coords_evaluation, dose_evaluation,
    dose_thresholds, distance_thresholds, 
    max_gamma=1.0001, skip_once_passed=True, quiet=True)

In [ ]:
dose_thresholds = np.arange(0.5,3,0.2)
distance_thresholds = np.arange(0.5,4,0.2)

print("Num Gamma threshold combos = {}".format(len(dose_thresholds) * len(distance_thresholds)))

In [ ]:
%%timeit

gamma = gamma_shell(
    coords_reference, dose_reference,
    coords_evaluation, dose_evaluation,
    dose_thresholds, distance_thresholds, 
    max_gamma=1.0001, skip_once_passed=True)

In [ ]:
gamma = gamma_shell(
    coords_reference, dose_reference,
    coords_evaluation, dose_evaluation,
    [1, 0.2], [1, 4], 
    max_gamma=1.1, skip_once_passed=True, local_gamma=True, quiet=True, interp_fraction=10)

baseline = {
    (1, 1): 96.9,
    (1, 4): 99.9,
    (0.2, 1): 91.8,
    (0.2, 4): 99.6
}

for key, value in gamma.items():
    print(baseline[key])
    print(convert_to_percent_pass(value))

In [ ]:
8.861 / 0.122

In [ ]:
def to_be_profiled():
    gamma_shell(
        coords_reference, dose_reference,
        coords_evaluation, dose_evaluation,
        dose_thresholds, distance_thresholds, 
        max_gamma=1.0001, skip_once_passed=True, quiet=True)
    
%prun to_be_profiled()

In [ ]:
# gamma = gamma_shell(
#     coords_reference, dose_reference,
#     coords_evaluation, dose_evaluation,
#     dose_thresholds, distance_thresholds, 
#     max_gamma=1.0001, skip_once_passed=True, quiet=True)

In [ ]:
9.76 / 0.745

In [ ]:
pass_rates = []

for key, value in gamma.items():
    pass_rates.append(convert_to_percent_pass(value))
    
pass_rates = np.reshape(pass_rates, (len(dose_thresholds), len(distance_thresholds)))
    
plt.contourf(distance_thresholds, dose_thresholds, pass_rates, 100, vmin=90, vmax=100, cmap='coolwarm_r')

plt.title('Gamma pass-rate space')
plt.xlabel('Distance Threshold (mm)')
plt.ylabel('Dose Threshold (%)')
plt.colorbar(label='Gamma pass rate (gamma <= 1)')

In [ ]:
def get_dummy_gamma_set():
    grid_x = np.arange(0, 1, 0.1)
    grid_y = np.arange(0, 1.2, 0.1)
    grid_z = np.arange(0, 1.4, 0.1)
    dimensions = (len(grid_x), len(grid_y), len(grid_z))
    coords = (grid_x, grid_y, grid_z)

    reference = np.zeros(dimensions)
    reference[3:-2:, 4:-2:, 5:-2:] = 1.015

    evaluation = np.zeros(dimensions)
    evaluation[2:-2:, 2:-2:, 2:-2:] = 1

    expected_gamma = np.zeros(dimensions)
    expected_gamma[2:-2:, 2:-2:, 2:-2:] = 0.4
    expected_gamma[3:-3:, 3:-3:, 3:-3:] = 0.7
    expected_gamma[4:-4:, 4:-4:, 4:-4:] = 1
    expected_gamma[3:-2:, 4:-2:, 5:-2:] = 0.5

    return coords, reference, evaluation, expected_gamma

In [ ]:
def test_regression_of_gamma_3d():
    """Test for changes in expected 3D gamma."""
    coords, reference, evaluation, expected_gamma = get_dummy_gamma_set()

    dose_thresholds = np.arange(1,3,0.5)
    distance_thresholds = np.arange(0.1,0.35,0.05)
    
    gamma3d = gamma_shell(
        coords, reference,
        coords, evaluation,
        dose_thresholds, distance_thresholds, lower_percent_dose_cutoff=0, quiet=True)

#     assert np.all(expected_gamma == gamma3d)

In [ ]:
%prun test_regression_of_gamma_3d()